In [1]:
import import_ipynb
import sys
import torch
from torch import nn
from torch import optim

if torch.cuda.is_available():
    cuda = torch.cuda.device("cuda:0")

%cd /home/ankit/Commitments/earthquake-prediction/

from preprocessing import *
from training import *
from postprocessing import *
from testing import *

%cd /home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi/

/home/ankit/Commitments/earthquake-prediction
importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from training.ipynb
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 90])
importing Jupyter notebook from postprocessing.ipynb
importing Jupyter notebook from testing.ipynb
/home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi


In [2]:
# SETTINGS D

data_file = "data.csv"
time_parser = time_from_string_dmy
mag_range = (5.0, 8.0)

rows = 3
cols = 3
total_params = rows*cols
time_intervals = 1200
train_size = 0.8
lookback_window = 80

In [3]:
earthquakes = extract_from_csv(data_file, time_parser, mag_range)
cells = Spatio_Temporal_Segmentation(rows, cols, time_intervals, earthquakes)

p = 0
n = 0
for i in range(time_intervals):
    for j in range(rows):
        for k in range(cols):
            if cells[i][j][k].has_event == 1:
                p += 1
            else:
                n += 1
print(p, n)

3317 7483


In [4]:
generator = TensorGen(cells)

eventtensor = generator.eventNoEvent()
freqtensor = generator.freq()
minmagtensor = generator.min_magnitude()
maxmagtensor = generator.max_magnitude()

if torch.cuda.is_available:
    eventtensor = eventtensor.cuda()
    freqtensor = freqtensor.cuda()
    minmagtensor = minmagtensor.cuda()
    maxmagtensor = maxmagtensor.cuda()
    
print(eventtensor.size())
print(freqtensor.size())
print(minmagtensor.size())
print(maxmagtensor.size())

torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])


<strong>TRAINING EVENT PREDICTOR</strong>

In [5]:
#SETTINGS M1

hidden_parameters = 40
linear_parameters = 40
num_layers = 2

In [6]:
model = LSTM(total_params, hidden_parameters, linear_parameters, total_params, num_layers=num_layers, activation=nn.Tanh())

if torch.cuda.is_available():
    model = model.cuda()

    
# out = model(eventtensor)
# print(out)

In [7]:
# SETTINGS T1
pos_weight = torch.tensor([2])
pos_weight = pos_weight.cuda()
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# loss_function = nn.MSELoss()
n_epochs = 100

In [12]:
optimizer = optim.Adagrad(model.parameters(), lr=0.001)

# tqdm._instances.clear()
train(model, eventtensor, optimizer, loss_function, n_epochs=1)

100%|██████████| 1150/1150 [00:45<00:00, 25.16it/s]


Epoch 1 completed. 
Training Loss: 0.766
Test Loss: 0.758


In [13]:
evaluate(data=eventtensor, model=model, train_size=0.8, lookback_window=lookback_window, threshold=0.5)

100%|██████████| 1120/1120 [00:24<00:00, 46.63it/s]


Evaluation results for training dataset:
7920 evaluations in total.
1177/2523 cells with earthquake activity were correctly identified. (46.65 %)
4814/5397 cells with no earthquake activity were correctly identified. (89.20 %)
583/1760 positives turned out to be false.
1346/6160 negatives turned out to be false.
Preicision: 0.669
Recall: 0.467
F1 Score: 0.550
Overall accuracy: 75.64 %
Evaluation results for training dataset:
2160 evaluations in total.
294/623 cells with earthquake activity were correctly identified. (47.19 %)
1351/1537 cells with no earthquake activity were correctly identified. (87.90 %)
186/480 positives turned out to be false.
329/1680 negatives turned out to be false.
Preicision: 0.613
Recall: 0.472
F1 Score: 0.533
Overall accuracy: 76.16 %
